In [1]:
import os
import anndata
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
import seaborn as sns 
import scipy
import csv
import gzip
import anndata as ad
from pathlib import Path
import glob

In [2]:
#Resource
AUXILLIARIES_FOLDERNAME = "/home/jing/pySCENIC/resources/"
HUMAN_TFS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'allTFs_mm.txt')

RANKING_DBS_FNAMES = list(map(lambda fn: os.path.join(AUXILLIARIES_FOLDERNAME, fn),
                       ['hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather',
                       'hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather']))

MOTIF_ANNOTATIONS_FNAME = os.path.join(AUXILLIARIES_FOLDERNAME, 'motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl')


In [3]:
#Outputs
DATASET_ID = 'GBM'
RESULTS_FOLDERNAME = "/home/jing/Phd_project/project_GBM/gbm_OUTPUT/gbm_OUTPUT_publication/gbm_OUT_publication_scenic_run3/"
os.chdir(RESULTS_FOLDERNAME)
F_LOOM_SCE =  os.path.join(RESULTS_FOLDERNAME, '{}_filtered_scenic.loom'.format(DATASET_ID))
f_pyscenic_output = os.path.join(RESULTS_FOLDERNAME, '{}_final_scenic.loom'.format(DATASET_ID))


ADJACENCIES_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.adjacencies.tsv'.format(DATASET_ID))
MOTIFS_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.motifs.csv'.format(DATASET_ID))

In [4]:
REGULONS_DAT_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.regulons.dat'.format(DATASET_ID))
AUCELL_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.auc.csv'.format(DATASET_ID))
BIN_MTX_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.bin.csv'.format(DATASET_ID))
THR_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.thresholds.csv'.format(DATASET_ID))

### Adata

In [5]:
directory = '/home/jing/Phd_project/project_GBM/gbm_DATA/gbm_DATA_GSE174554/gbm_DATA_scRNA_atlas'
os.chdir(directory)
names_list=['GSM5319518_SF2777','GSM5319548_SF2979','GSM5319519_SF2990',
                'GSM5319549_SF3073','GSM5319520_SF3076','GSM5319550_SF3243',
                'GSM5319521_SF3391','GSM5319551_SF3448','GSM5319511_SF11916',
                'GSM5319543_SF12382','GSM5319506_SF11082','GSM5319562_SF11488',
                'GSM5319530_SF9358','GSM5319568_SF9962','GSM5319559_SF9798','GSM5319532_SF9494']
adata_list = []

# Loop over each sample and read in the AnnData object
for name in names_list:

    mtx =f"{name}_matrix.mtx.gz"
    adata = sc.read_mtx(mtx)
    cells=pd.read_csv(f'{name}_barcodes.tsv.gz',header=None)
    features=pd.read_csv(f'{name}_features.tsv.gz',header=None,sep='\t')
    adata= adata.T
    #check the columns first to make sure they are the ones you need 
    adata.obs['CellID']= cells[0].tolist()
    adata.obs.index = adata.obs['CellID']
    adata.var['Gene']= features[0].tolist()
    adata.var.index= adata.var['Gene']
    adata.var_names_make_unique() 
    adata.var['mt'] =adata.var_names.str.startswith('MT-')


    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.calculate_qc_metrics(adata,qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    adata= adata[adata.obs.n_genes_by_counts <6000, :]
    adata= adata[adata.obs.pct_counts_mt< 5, :].copy()


    adata.obs['source'] = name[11:]
    adata.layers["counts"] = adata.X.copy()    
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata  # keep full dimension safe
    adata_list.append(adata)
batch_names = [adata.obs['source'].iloc[0] for adata in adata_list]
adata = adata_list[0].concatenate(adata_list[1:], batch_key='source', batch_categories=batch_names) 
adata.obs.index.name = "ref"
display(adata.obs)

/tmp/ipykernel_1100579/3309545611.py:40: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata = adata_list[0].concatenate(adata_list[1:], batch_key='source', batch_categories=batch_names)


,CellID,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,source
ref,,,,,,,
AAACCCAAGGGATCAC-1-SF2777,AAACCCAAGGGATCAC-1,468,468,579.0,9.0,1.554404,SF2777
AAACCCAGTCGATTTG-1-SF2777,AAACCCAGTCGATTTG-1,311,311,402.0,4.0,0.995025,SF2777
AAACCCAGTCGTCAGC-1-SF2777,AAACCCAGTCGTCAGC-1,864,864,1518.0,3.0,0.197628,SF2777
AAACCCAGTTGTAAAG-1-SF2777,AAACCCAGTTGTAAAG-1,632,632,891.0,12.0,1.346801,SF2777
AAACCCATCTATCGGA-1-SF2777,AAACCCATCTATCGGA-1,849,849,1338.0,43.0,3.213752,SF2777
...,...,...,...,...,...,...,...
TTTGGTTTCATTATCC-1-SF9494,TTTGGTTTCATTATCC-1,2027,2027,4502.0,0.0,0.000000,SF9494
TTTGGTTTCCCTCGTA-1-SF9494,TTTGGTTTCCCTCGTA-1,1972,1972,4071.0,0.0,0.000000,SF9494
TTTGTTGTCAGACAAA-1-SF9494,TTTGTTGTCAGACAAA-1,2340,2340,5272.0,0.0,0.000000,SF9494


In [6]:
outdir='/home/jing/Phd_project/project_GBM/gbm_OUTPUT/gbm_OUTPUT_publication'
doublet_df = pd.read_csv(os.path.join(outdir,'doublet_predictions.csv'), index_col=0)
doublet_df

for i in adata.obs.index:
    adata.obs.loc[i,'solo'] = doublet_df.loc[i,'Solo_Prediction']
adata_singlet = adata[adata.obs['solo']=='singlet']
adata_singlet

View of AnnData object with n_obs × n_vars = 50738 × 33694
    obs: 'CellID', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'source', 'solo'
    var: 'Gene', 'mt', 'n_cells_by_counts-SF11082', 'mean_counts-SF11082', 'pct_dropout_by_counts-SF11082', 'total_counts-SF11082', 'n_cells_by_counts-SF11488', 'mean_counts-SF11488', 'pct_dropout_by_counts-SF11488', 'total_counts-SF11488', 'n_cells_by_counts-SF11916', 'mean_counts-SF11916', 'pct_dropout_by_counts-SF11916', 'total_counts-SF11916', 'n_cells_by_counts-SF12382', 'mean_counts-SF12382', 'pct_dropout_by_counts-SF12382', 'total_counts-SF12382', 'n_cells_by_counts-SF2777', 'mean_counts-SF2777', 'pct_dropout_by_counts-SF2777', 'total_counts-SF2777', 'n_cells_by_counts-SF2979', 'mean_counts-SF2979', 'pct_dropout_by_counts-SF2979', 'total_counts-SF2979', 'n_cells_by_counts-SF2990', 'mean_counts-SF2990', 'pct_dropout_by_counts-SF2990', 'total_counts-SF2990', 'n_cells_by_counts-SF3073', 'mean_counts-SF3073

In [7]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata_singlet.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata_singlet.obs_names) ,
    "nGene": np.array( np.sum(adata_singlet.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata_singlet.X.transpose() , axis=0)).flatten() ,
}
lp.create( F_LOOM_SCE, adata_singlet.X.transpose(), row_attrs, col_attrs)

BlockingIOError: [Errno 11] Unable to synchronously create file (unable to lock file, errno = 11, error message = 'Resource temporarily unavailable')

In [ ]:
os.chdir(RESULTS_FOLDERNAME)

### SECNIC inference

In [ ]:
#I) GRN
!pyscenic grn {F_LOOM_SCE} {HUMAN_TFS_FNAME} -o {ADJACENCIES_FNAME} --num_workers 20